# Task 2:

The following code can be used in python to determine the machine accuracy ε (or Machine epsilon):

```
import numpy as np
numprec=1.0
while 1<np.float32(1+numprec): # Check sum with single precision
# while 1<1+numprec: # Check sum with double precision
numprec*=0.5
numprec*=2
```

1. Determine ε using the above code, both for single and double precision. A double precision number
uses 8 bytes of storage, whereas a single precision number only occupies 4 bytes.
2. Give a definition of the machine accuracy based on the code above (use words and not mathematical
expression)

In [9]:
import numpy as np

# Divide 1 by 2 repeatedly until it is equivalent to 0 according to 32 bit floating point values
numprec_single = 1.0
while 1<np.float32(1+numprec_single): #single
    numprec_single*=0.5
numprec_single*=2
print(f'Machine epsilon (Single, 4 bytes):{numprec_single}')

# Divide 1 by 2 repeatedly until it is equivalent to 0 according to a double floating point value
numprec_double = 1.0
#while 1<np.float32(1+numprec): #single
while 1<1+numprec_double: #double
    numprec_double*=0.5
numprec_double*=2
print(f'Machine epsilon (Double, 8 bytes):{numprec_double}')

Machine epsilon (Single, 4 bytes):1.1920928955078125e-07
Machine epsilon (Double, 8 bytes):2.220446049250313e-16
